In [ ]:
from handler_demographic import merge_demographic
from handler_GIS_demographic import create_basic_layers,create_demographic_sa, add_settlements_data
# from general_functions import to_df
from handler_GIS_lines import Lines_in_JLM_metropolin
from handler_rishui import rishui_basic_data, rishui_resolution3, passengers,clean_ebitzua, ebitzua_resolutions
from handler_complaints import main_complaints

In [2]:
#create demographic table and save as csv for transferring to arcgis 
demographic_table=merge_demographic() 

In [3]:
create_basic_layers()
create_demographic_sa()
add_settlements_data()

statisticalareas_2022 layer created
jlm_metropolitan_boundries layer created
jlm_city_boundries layer created
settlements layer created
bus_routes_from_GTFS layer created
stops_from_GTFS layer created
statistical areas with demographic date is printed


In [4]:
JLM_lines=Lines_in_JLM_metropolin(threshold_value=0.25)


JLM metropolitan lines list and layer are printed


In [6]:
risui_res12=rishui_basic_data(JLM_lines)
rishui_res3=rishui_resolution3(risui_res12)

rishui data res12 is printed
rishui data res3 is printed


In [7]:
passengers_res12, passengers_res3=passengers()

passengers numbers per line are printed


In [8]:
ebtzua=clean_ebitzua()
x=ebitzua_resolutions(ebtzua)

In [2]:
complaints_res1,complaints_res2, complaints_res3=main_complaints()

In [10]:
statistical_areas_2022 = 'external_files/gis_layers/statisticalareas_demography2022.gdb/statisticalareas_demography2022'
jlm_metro= 'external_files/gis_layers/statisticalareas_demography2022.gdb/jlm_metro'

In [ ]:
arcpy.management.MakeFeatureLayer(statistical_areas_2022, "statisticalareas_2022")
arcpy.management.MakeFeatureLayer(jlm_metro, "jlm_metropolitan_boundries")


<Result 'statisticalareas_2022'>

<Result 'true'>

In [13]:
arcpy.management.MakeFeatureLayer(statistical_areas_2022, "statisticalareas_2022")


NameError: name 'statistical_areas_2022' is not defined

In [14]:
arcpy.Describe('statisticalareas_2022').path

'c:\\thesis\\gis\\scripts\\thesis_code\\external_files\\gis_layers\\statisticalareas_demography2022.gdb'

In [12]:
arcpy.Describe('jlm_metropolitan_boundries').path

'c:\\thesis\\gis\\scripts\\thesis_code\\external_files\\gis_layers\\statisticalareas_demography2022.gdb'

In [10]:
to_df('sa_jlm_metro')

,OBJECTID,Shape,SEMEL_YESHUV,STAT11,YISHUV_STAT11,Shem_Yishuv,Shem_Yishuv_English,Stat11_Unite,Stat11_Ref,Main_Function_Code,...,age_75_79,age_80_84,age_85_up,AREA,Shape_Length,Shape_Area,socioeco_2019,pop_2019,RANK_2019,updated_ultraorthodox_percent
0,1,"(192782.40065588, 625477.7245936742)",27,1.0,270001,שדות מיכה,SEDOT MIKHA,1,None,1.0,...,9.0,6.0,8.0,None,4488.196389,923522.463106,NaN,378.0,235.0,0.0
1,2,"(214997.3500377348, 633472.4243585464)",208,1.0,2080001,מוצא עילית,MOZA ILLIT,1,None,1.0,...,91.0,50.0,86.0,None,3352.923048,548118.036937,NaN,1211.0,924.0,0.0
2,3,"(220625.44780368585, 629935.9393509859)",3000,1313.0,30001313,ירושלים,JERUSALEM,1313,None,1.0,...,100.0,80.0,102.0,None,2088.826758,188617.798467,7.0,1974.0,995.0,12.0
3,4,"(220910.92075865614, 630332.388663023)",3000,1314.0,30001314,ירושלים,JERUSALEM,1314,None,1.0,...,125.0,75.0,109.0,None,2804.715170,409097.608181,7.0,1963.0,1237.0,0.0
4,5,"(220872.55106941046, 627384.0131515897)",127,1.0,1270001,רמת רחל,RAMAT RAHEL,1,None,1.0,...,15.0,9.0,9.0,None,3591.744644,743360.785709,NaN,521.0,115.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,378,"(229386.37849999964, 628933.5845999996)",3781,1.0,37810001,קדר,QEDAR,1,None,1.0,...,5.0,NaN,NaN,None,314.159265,7853.981634,NaN,1598.0,455.0,0.0
378,379,"(209638.05829999968, 618425.1920999996)",3794,1.0,37940001,בת עין,BAT AYIN,1,None,1.0,...,7.0,5.0,6.0,None,314.159265,7853.981634,NaN,1567.0,62.0,17.1
379,380,"(203623.21270000003, 647427.4465999994)",3796,1.0,37960001,כפר האורנים,KEFAR HAORANIM,1,None,1.0,...,13.0,4.0,5.0,None,314.159265,7853.981634,NaN,2657.0,927.0,0.0
380,381,"(201755.6989000002, 648472.0160000008)",3823,1.0,38230001,גני מודיעין,GANNE MODI'IN,1,None,1.0,...,29.0,18.0,6.0,None,314.159265,7853.981634,NaN,2603.0,110.0,56.7


In [ ]:
layername='GeocodeDonationLocations'
layerfile1=arcpy.management.MakeFeatureLayer(ResultPath1,layername)[0]  # get the layer object
# Or 
# layerfile1=arcpy.management.MakeFeatureLayer(ResultPath1,layername).getOutput(0)  # get the layer object
p=arcpy.mp.ArcGISProject(r"~filepath ~to ~.aprx")
m=p.listMaps()[0]
m.addLayer(layerfile1,"TOP")
p.save()

<Result 'C:/thesis/gis/thesis/thesis.gdb/demographic_table'>

In [2]:
urb()

,CODE OF LOCALITY,name_hebrew,name_english,pop_2019,RANK_2019,socioeco_2019,YISHUV_STAT11
0,31,אופקים,OFAQIM,6123,106,1,31000006
1,31,אופקים,OFAQIM,3044,254,3,31000003
2,31,אופקים,OFAQIM,3043,271,3,31000004
3,31,אופקים,OFAQIM,2773,375,4,31000002
4,31,אופקים,OFAQIM,5246,376,4,31000001
...,...,...,...,...,...,...,...
1630,5000,תל אביב -יפו,TEL AVIV - YAFO,2559,1629,10,50000131
1631,5000,תל אביב -יפו,TEL AVIV - YAFO,2543,1630,10,50000437
1632,5000,תל אביב -יפו,TEL AVIV - YAFO,954,1631,10,50000914
1633,5000,תל אביב -יפו,TEL AVIV - YAFO,2134,1633,10,50000224


In [3]:
reg('sub')

,CODE OF LOCALITY,name_hebrew,pop_2019,RANK_2019,name_english,YISHUV_STAT11
1,1192,ערערה-בנגב,18331,2,AR'ARA-BANEGEV,11920001
2,1054,תל שבע,20808,3,TEL SHEVA,10540001
3,1059,כסיפה,21808,4,KUSEIFE,10590001
5,3797,מודיעין עילית,76373,6,MODI'IN ILLIT,37970001
6,1303,חורה,22337,7,HURA,13030001
...,...,...,...,...,...,...
250,1224,כוכב יאיר,8702,251,KOKHAV YA'IR,12240001
251,666,עומר,7530,252,OMER,6660001
252,1271,להבים,6414,253,LEHAVIM,12710001
253,267,כפר שמריהו,1905,254,KEFAR SHEMARYAHU,2670001


In [5]:
x=reg('reg')
x

,CODE OF LOCALITY,name_hebrew,name_english,pop_2019,RANK_2019,YISHUV_STAT11
0,528,עוזייר,UZEIR,3276,22.0,5280001
1,539,רומאנה,RUMMANE,1212,27.0,5390001
2,997,רומת הייב,RUMAT HEIB,2046,36.0,9970001
3,993,חמאם,HAMAM,1503,84.0,9930001
4,1346,תרבין א-צאנע (יישוב),TARABIN AS-SANI,1138,3.0,13460001
...,...,...,...,...,...,...
990,692,שפיר,SHAFIR,920,437.0,6920001
991,2042,עין גדי,EN GEDI,610,296.0,20420001
992,1261,הר עמשא,HAR AMASA,198,422.0,12610001
993,1124,נאות הכיכר,NE'OT HAKIKKAR,447,678.0,11240001


In [4]:
import pandas as pd
x="external_files/cbs_files/socio_eco_urb.xlsx"
pd.read_excel(x)

,MUNICIPAL STATUS,CODE OF LOCALITY,שם יישוב,NAME OF LOCALITY,CLUSTER 2019[4],CLUSTER 2017[4],CODE OF STATISTICAL AREA,INDEX POPULATION 2019[1],INDEX VALUE 2019[2],RANK 2019[3],CLUSTER 2019[4] .1,CLUSTER 2017[4].1
0,0,31,אופקים,OFAQIM,3,3,6,6123.034701,-1.723285,106,1,1
1,0,31,אופקים,OFAQIM,3,3,3,3044.829053,-0.964277,254,3,3
2,0,31,אופקים,OFAQIM,3,3,4,3043.845597,-0.908773,271,3,3
3,0,31,אופקים,OFAQIM,3,3,2,2773.050577,-0.541246,375,4,3
4,0,31,אופקים,OFAQIM,3,3,1,5246.151512,-0.539951,376,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...
1630,0,5000,תל אביב -יפו,TEL AVIV - YAFO,8,8,131,2559.804613,2.363704,1629,10,10
1631,0,5000,תל אביב -יפו,TEL AVIV - YAFO,8,8,437,2543.218005,2.368478,1630,10,10
1632,0,5000,תל אביב -יפו,TEL AVIV - YAFO,8,8,914,954.484779,2.400347,1631,10,10
1633,0,5000,תל אביב -יפו,TEL AVIV - YAFO,8,8,224,2134.774259,2.518987,1633,10,10
